In [ ]:
# Add the directory to sys.path
import sys
sys.path.append('.')

# Now you can import the module
from utils.tle_processing import *


In [ ]:
import numpy as np
import spacetrack.operators as op
from datetime import datetime,timedelta
import pandas as pd
from spacetrack import SpaceTrackClient
from io import StringIO  # Import StringIO
import time
import matplotlib.pyplot as plt
import tqdm

In [ ]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [ ]:
# Save model
import joblib
import os

# Multi-output models
- Vector Autoregression
- RNN (TF)
- MultiOutput Regressor

In [ ]:
resample_tle_df = pd.read_csv('sample_dataset/sample_TLE_40055.csv')
resample_tle_df.set_index('Epoch', inplace=True)
resample_tle_df.head()

In [ ]:
resample_tle_df.columns

In [ ]:
# --- 1. Prepare Sample Data ---
data_for_training = resample_tle_df.copy()

# Add_diff
# data_for_training['diff_eccentricity'] = data_for_training['Eccentricity'].diff()
data_for_training['diff_revs_number'] =  data_for_training['Revolution Number at Epoch'].diff()
data_for_training.head()


In [ ]:
# data_for_training['First Derivative Mean Motion']

In [ ]:
cols_for_training = ['First Derivative Mean Motion', 
       'Mean Motion (revolutions per day)', 
       'Eccentricity',  'diff_revs_number', 'Inclination (degrees)',
       'Right Ascension of the Ascending Node (degrees)',
       'Argument of Perigee (degrees)', 'Mean Anomaly (degrees)']

# Avoid near zero values 
# data_for_training['First Derivative Mean Motion'] = data_for_training['First Derivative Mean Motion']*10000
# data_for_training['Eccentricity'] = data_for_training['Eccentricity']*10000



data = data_for_training[cols_for_training].copy()
data.head()

In [ ]:
data.describe()

## Statistical testing

In [ ]:
from statsmodels.tsa.stattools import adfuller, kpss

# ADF : unit root test ==> null hypothesis is non-stationary
# Kwiatkowski–Phillips–Schmidt–Shin (KPSS) Test ==> null hypothesis is stationary

print('At confidence level 90%')

for col in data.columns:
    col_series = data[col]
    
    print(f"Features : {col}")
    # Perform the ADF test
    adf_result = adfuller(col_series)
    #     print("ADF Statistic:", adf_result[0])
    #     print("p-value:", adf_result[1])
    #     print("Critical Values:", adf_result[4])
    
    if adf_result[1] < 0.1:
        print(f'p-value: {adf_result[1]} , Reject null for ADF: Stationary')
    else:
        print(f'p-value: {adf_result[1]} , Accept null for ADF: Non-stationary')
        
    # Perform the KPSS test
    #     kpss_result = kpss(col_series)

    #     if kpss_result[1] < 0.05:
    #         print('Reject null for KPSS: Non-stationary')
    #     else:
    #         print('Accept null for KPSS: Stationary')
    
    print('=='*30)

In [ ]:
plt.plot(data['Mean Anomaly (degrees)'])

In [ ]:
# --- 2. Data Preparation for Multi-Output Forecasting ---
# We will create a dataset where the model predicts the next 1 time steps
# using the previous 10 time steps.
look_back = 2 # current + 2 records
forecast_horizon = 1
features = []
targets = []
num_samples = data_for_training.shape[0]

In [ ]:
for i in range(num_samples - look_back - forecast_horizon + 1):
    # The input features are the last `look_back` values of all series
    X_window = data.iloc[i : i + look_back].values
    
    # The target variables are the next `forecast_horizon` values of all series
    y_window = data.iloc[i + look_back : i + look_back + forecast_horizon].values
    
    # Flatten the windows to create a single row for the features and targets
    features.append(X_window.flatten())
    targets.append(y_window.flatten())

X = np.array(features)
y = np.array(targets)

print(f"Shape of X (input features): {X.shape}")
print(f"Shape of y (target variables): {y.shape}")

In [ ]:
# --- 3. Scaling the Data ---
# It's good practice to scale the data, especially for models that are
# sensitive to feature magnitudes. We use a MinMaxScaler.
# We need to scale X and y separately, and use a different scaler for each
# to allow for inverse transformation later.

# X_scaler = MinMaxScaler()
# y_scaler = MinMaxScaler()

X_scaler = StandardScaler()
y_scaler = StandardScaler()

X_scaled = X_scaler.fit_transform(X)
y_scaled = y_scaler.fit_transform(y)

# --- 4. Splitting the Data ---
# For time series, we split chronologically. The last 20% of the data
# will be used for testing.
split_index = int(len(X_scaled) * 0.8)
X_train, X_test = X_scaled[:split_index], X_scaled[split_index:]
y_train, y_test = y_scaled[:split_index], y_scaled[split_index:]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
# --- 5. Model Training ---
# We instantiate a base estimator (RandomForestRegressor) and then wrap it
# with the MultiOutputRegressor. This allows the model to predict all
# output variables simultaneously.
base_estimator = RandomForestRegressor(n_estimators=100, random_state=42)
model = MultiOutputRegressor(base_estimator, n_jobs=-1)

# Train the model on the scaled training data
print("\nTraining the model...")
model.fit(X_train, y_train)
print("Model training complete.")

In [ ]:


# --- 6. Save the Trained Model ---
# We'll save the model and the scalers so we can use them later without
# retraining.
model_filename = 'multi_output_model.joblib'



# Your own path
save_path = ''


joblib.dump(model, os.path.join(save_path,model_filename))

print(f"\nModel and scalers saved to {model_filename},")

In [ ]:
# --- 6. Prediction and Inverse Scaling ---
# Make predictions on the test set. The predictions will be scaled.
y_pred_scaled = model.predict(X_test)

# Inverse transform the predictions and the actual test data to their original scale
y_pred_original = y_scaler.inverse_transform(y_pred_scaled)
y_test_original = y_scaler.inverse_transform(y_test)


In [ ]:
# --- 7. Evaluation and Visualization ---
# Reshape the data back into its original time series format for plotting and evaluation.
# The shape is (num_test_samples, forecast_horizon * num_series).
# We want to reshape it to (num_test_samples, forecast_horizon, num_series).
y_test_reshaped = y_test_original.reshape(y_test_original.shape[0], forecast_horizon, -1)
y_pred_reshaped = y_pred_original.reshape(y_pred_original.shape[0], forecast_horizon, -1)

In [ ]:
from sklearn.metrics import mean_squared_error


In [ ]:
# Calculate MAE and MAPE for each series over the entire test set
mae_per_series = []
mape_per_series = []
rmse_per_series = []
for i in range(y_test_reshaped.shape[2]):
    actual_series = y_test_reshaped[:, :, i]
    pred_series = y_pred_reshaped[:, :, i]
    
    # Calculate MAE
    mae = mean_absolute_error(actual_series, pred_series)
    mse = mean_squared_error(actual_series, pred_series)
    rmse = np.sqrt(mse)
    rmse_per_series.append(rmse)
    mae_per_series.append(mae)
    
    # Calculate a simple version of MAPE (Mean Absolute Percentage Error)
    # We use a small epsilon to avoid division by zero.
    # Note: MAPE is sensitive to zero or near-zero values.
    non_zero_actuals = actual_series[np.abs(actual_series) > 1e-6]
    if len(non_zero_actuals) > 0:
        percentage_error = np.mean(np.abs((pred_series[np.abs(actual_series) > 1e-6] - non_zero_actuals) / non_zero_actuals)) * 100
    else:
        percentage_error = np.nan
    mape_per_series.append(percentage_error)

print("\nMean Absolute Error (MAE) and Mean Absolute Percentage Error (MAPE) for each time series:")
for i in range(len(mae_per_series)):
#     print(f"  - Series {i+1} ({cols_for_training[i]}): MAE = {mae_per_series[i]:.4f}, MAPE = {mape_per_series[i]:.2f}%")
#     print(f"  - Series {i+1} ({cols_for_training[i]}): MAE = {mae_per_series[i]:.4f}")
    print(f"  - Series {i+1} ({cols_for_training[i]}): RMSE = {rmse_per_series[i]:.4f} :MAE = {mae_per_series[i]:.9f}, MAPE = {mape_per_series[i]:.2f}%")


In [ ]:
y_test_reshaped.shape

In [ ]:
# Plot the predictions vs. actual values for the first time series

for i in range(len(cols_for_training)):
    plt.figure(figsize=(15, 6))
    plt.title(f"Figure {i} {cols_for_training[i]}: Actual vs. Predicted (Test Set)")
    
    if cols_for_training[i] == 'diff_revs_number':
        plt.plot(y_test_reshaped[:, 0, i].cumsum(), label='Actual', color='blue')
        plt.plot(y_pred_reshaped[:, 0, i].cumsum(), label='Predicted', color='red', linestyle='--')

    else:
        plt.plot(y_test_reshaped[:, 0, i], label='Actual', color='blue')
        plt.plot(y_pred_reshaped[:, 0, i], label='Predicted', color='red', linestyle='--')
        
#     plt.plot(y_test_reshaped[:, 0, i], label='Actual', color='blue')
#     plt.plot(y_pred_reshaped[:, 0, i], label='Predicted', color='red', linestyle='--')
    plt.legend()
    plt.grid(True)
    plt.xlabel("Test Sample")
    plt.ylabel(f"{cols_for_training[i]} Value")
    plt.show()

# Hyperparameters tuning

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV

In [ ]:
# --- 5. Model Training and Hyperparameter Tuning ---
# We will now use GridSearchCV to find the best hyperparameters for multiple models.
# The best model will be selected for evaluation.

models = {
    'RandomForestRegressor': {
        'estimator': RandomForestRegressor(random_state=42),
        'param_grid': {'estimator__n_estimators': [50, 100, 200], 'estimator__max_depth': [None, 5, 10]}
    },
    'KNeighborsRegressor': {
        'estimator': KNeighborsRegressor(),
        'param_grid': {'estimator__n_neighbors': [3, 5, 7], 'estimator__weights': ['uniform', 'distance']}
    },
    'GradientBoostingRegressor': {
        'estimator': GradientBoostingRegressor(random_state=42),
        'param_grid': {'estimator__n_estimators': [50, 100], 'estimator__learning_rate': [0.05, 0.1]}
    },
    # SVR can be very slow, so we use a small, simple grid.
    'SVR': {
        'estimator': SVR(),
        'param_grid': {'estimator__kernel': ['rbf'], 'estimator__C': [0.1, 1, 10]}
    }
}

best_model = None
best_score = -np.inf
best_model_name = ""

print("\nStarting Hyperparameter Tuning...")

for name, model_data in models.items():
    print(f"\n--- Tuning {name} ---")
    base_estimator = model_data['estimator']
    param_grid = model_data['param_grid']
    
    # Wrap the base estimator in MultiOutputRegressor
    multi_output_estimator = MultiOutputRegressor(base_estimator, n_jobs=-1)
    
    # Set up GridSearchCV
    grid_search = GridSearchCV(multi_output_estimator, param_grid, cv=3, scoring='neg_mean_absolute_error', n_jobs=-1, verbose=1)
    
    # Fit the grid search on the training data
    grid_search.fit(X_train, y_train)
    
    print(f"Finished tuning {name}")
    print(f"Best parameters for {name}: {grid_search.best_params_}")
    print(f"Best cross-validation score (negative MAE) for {name}: {grid_search.best_score_:.4f}")
    
    # Check if this is the best model so far
    if grid_search.best_score_ > best_score:
        best_score = grid_search.best_score_
        best_model = grid_search.best_estimator_
        best_model_name = name

print(f"\nModel tuning complete. The overall best model is: {best_model_name}")

In [ ]:
model_filename = 'best_multi_output_model.joblib'
save_path = 'MultiOutputModel'


joblib.dump(best_model, os.path.join(save_path,model_filename))

In [ ]:

# --- 6. Prediction and Inverse Scaling ---
# Make predictions on the test set. The predictions will be scaled.
y_pred_scaled = best_model.predict(X_test)

# Inverse transform the predictions and the actual test data to their original scale
y_pred_original = y_scaler.inverse_transform(y_pred_scaled)
y_test_original = y_scaler.inverse_transform(y_test)

y_test_reshaped = y_test_original.reshape(y_test_original.shape[0], forecast_horizon, -1)
y_pred_reshaped = y_pred_original.reshape(y_pred_original.shape[0], forecast_horizon, -1)



In [ ]:
# Calculate MAE and MAPE for each series over the entire test set
mae_per_series = []
mape_per_series = []
for i in range(y_test_reshaped.shape[2]):
    actual_series = y_test_reshaped[:, :, i]
    pred_series = y_pred_reshaped[:, :, i]
    
    # Calculate MAE
    mae = mean_absolute_error(actual_series, pred_series)
    mae_per_series.append(mae)
    
    # Calculate a simple version of MAPE (Mean Absolute Percentage Error)
    # We use a small epsilon to avoid division by zero.
    # Note: MAPE is sensitive to zero or near-zero values.
    non_zero_actuals = actual_series[np.abs(actual_series) > 1e-6]
    if len(non_zero_actuals) > 0:
        percentage_error = np.mean(np.abs((pred_series[np.abs(actual_series) > 1e-6] - non_zero_actuals) / non_zero_actuals)) * 100
    else:
        percentage_error = np.nan
    mape_per_series.append(percentage_error)

print("\nMean Absolute Error (MAE) and Mean Absolute Percentage Error (MAPE) for each time series:")
for i in range(len(mae_per_series)):
#     print(f"  - Series {i+1} ({cols_for_training[i]}): MAE = {mae_per_series[i]:.4f}, MAPE = {mape_per_series[i]:.2f}%")
    print(f"  - Series {i+1} ({cols_for_training[i]}): MAE = {mae_per_series[i]:.4f}")
